In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-12"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16472,2024-12-12,Brasil Nbb,19:30,Paulistano,Caxias do Sul,1.11,5.17,147.5,1.84,1.82,-13.5,2.02,1.68,pna3wHlS,0.900901,0.193424,0.543478,0.549451,0.094324,0.6,0.6,NaN,NaN,113.312,40.203399,0.354803,248.250,100.400584,0.404433,104.482,251.362,112.32,392.63,0.0,0.0,0.0,0.0,0.914285,0.007728,0.129955,-2.44,-0.488,-0.225410,0.677542,0.4,-0.277542,-2.79,-0.558,-7.473118,0.292264,0.2,-0.092264
16473,2024-12-12,Brasil Nbb,19:30,R10 Score Vasco da Gama,Brasília,1.52,2.30,154.5,1.81,1.85,-5.5,2.05,1.65,4WuzYfCk,0.657895,0.434783,0.552486,0.540541,0.092677,0.4,0.6,NaN,NaN,196.428,45.710299,0.232708,268.152,221.331444,0.825395,138.688,353.310,238.95,630.00,0.0,1.0,0.0,0.0,0.288766,0.015456,0.152888,-0.93,-0.186,-2.795699,0.662838,0.7,0.037162,2.29,0.458,2.838428,0.311236,0.4,0.088764
16474,2024-12-12,Brasil Nbb,20:00,Corinthians Paulista,Fortaleza B.C.,1.29,3.15,155.5,1.85,1.81,-8.5,1.94,1.73,xpvrWYt2,0.775194,0.317460,0.540541,0.552486,0.092654,0.6,0.4,NaN,NaN,157.402,37.740970,0.239774,213.126,137.859215,0.646844,141.982,262.576,93.84,442.33,0.0,0.0,0.0,0.0,0.592441,0.015456,0.080922,-3.48,-0.696,-0.416667,0.567580,0.1,-0.467580,-5.00,-1.000,-2.150000,0.387949,0.3,-0.087949
16475,2024-12-12,Brasil Nbb,20:00,Pinheiros,Unifacisa,1.59,2.15,151.5,1.87,1.79,-4.5,2.05,1.65,hhMUgxZ8,0.628931,0.465116,0.534759,0.558659,0.094047,0.4,0.2,NaN,NaN,209.492,59.745748,0.285193,139.500,73.134534,0.524262,157.562,190.150,248.82,265.54,0.0,0.0,0.0,0.0,0.211754,0.030912,0.152888,-0.46,-0.092,-6.413043,0.494040,0.4,-0.094040,-1.26,-0.252,-4.563492,0.427883,0.2,-0.227883
16476,2024-12-12,Brasil Nbb,20:00,São Paulo,União Corinthians,1.35,2.86,162.5,1.84,1.82,-7.5,1.94,1.74,bkPxhG3L,0.740741,0.349650,0.543478,0.549451,0.090391,0.6,0.8,NaN,NaN,128.100,52.494937,0.409797,175.686,23.701962,0.134911,137.200,235.534,99.68,162.50,0.0,0.0,0.0,0.0,0.507236,0.007728,0.076859,0.83,0.166,2.108434,0.591585,0.6,0.008415,-0.82,-0.164,-11.341463,0.329594,0.4,0.070406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16536,2024-12-12,Rússia Premier League Feminina,09:00,Enisey Krasnoyarsk F,MBA Moscow F,3.31,1.30,138.5,1.85,1.85,6.5,1.95,1.74,lOT7eyV2,0.302115,0.769231,0.540541,0.540541,0.071346,0.6,0.2,NaN,NaN,218.906,154.153250,0.704198,130.586,102.870954,0.787764,181.816,158.184,140.76,91.63,0.0,0.0,0.0,0.0,0.616609,0.000000,0.080484,-0.80,-0.160,-14.437500,0.591130,0.7,0.108870,2.64,0.528,0.568182,0.000000,0.0,0.000000
16537,2024-12-12,Rússia Premier League Feminina,10:00,Dynamo Moscow F,UMMC Ekaterinburg F,9.45,21.00,144.5,1.85,1.85,30.5,2.02,1.61,jeYdZpE0,0.105820,0.047619,0.540541,0.540541,-0.846561,0.0,0.8,NaN,NaN,311.596,217.989405,0.699590,98.308,12.643794,0.128614,0.000,532.228,155.22,100.88,0.0,0.0,0.0,0.0,0.536426,0.000000,0.159732,0.00,0.000,inf,0.000000,0.0,0.000000,23.08,4.616,4.332756,0.703398,0.9,0.196602
16538,2024-12-12,Rússia Premier League Feminina,11:00,Neftyanik Omsk F,Nika Siktivkar F,3.44,1.28,136.5,1.85,1.85,6.5,2.03,1.65,pjNGgFaF,0.290698,0.781250,0.540541,0.540541,0.071948,0.4,0.4,NaN,NaN,674.150,725.380097,1.075992,220.208,250.252944,1.136439,181.446,129.360,107.87,181.76,0.0,0.0,0.0,0.0,0.647182,0.000000,0.146033,-0.75,-0.150,-16.266667,0.000000,0.0,0.000000,-1.29,-0.258,-1.085271,0.574297,0.4,-0.174297
16539,202

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,16:00,Croácia Copa Da Croácia,Kvarner,Cedevita,160.5,1.84,0.999,Over
